<h1 align="center"> Principals of Biomedical Engineering </h1>
<h3 align="center"> Dr. M. Jahed </h3>
<br>
<h2 align="center"> EMG Classificatioh </h2>
<br>
<h4 align="center">Mohammad Hossein Shafieizadegan</h4>
<h4 align="center">Navid Bagheri Shouraki</h4>

# Add Required packages

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, iirnotch
from sklearn.preprocessing import StandardScaler

# Load Data

Load the dataset, whether our own acquired data or online free available dataset

# Signal Preprocessing

The following steps must be done:
- Bandpass Filtering
- Notch filter
- Z-score

### Bandpass filtering

We will utilize a bandpass filter regarding the frequency range of EMG signals

In [ ]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

In [ ]:
fs = 1000
f_low = 20
f_high = 200

emg_signal_bandpass = butter_bandpass_filter(emg_signal, f_low, f_high, fs)

### Notch filter
Applying `50 Hz` notch filter

In [ ]:
def notch_filter(data, notch_freq, fs, Q=30):
    nyq = 0.5 * fs
    f0 = notch_freq / nyq
    b, a = iirnotch(f0, Q)
    y = filtfilt(b, a, data)
    return y

In [ ]:
notch_freq = 50
emg_signal_notch = notch_filter(emg_signal_bandpass, notch_freq, fs)

### Z-score normalization

In [ ]:
# Step 3: Z-score normalization
scaler = StandardScaler()
emg_signal_normalized = scaler.fit_transform(emg_signal_notch.reshape(-1, 1)).flatten()

# Update the DataFrame with the preprocessed signal
df['EMG_signal_preprocessed'] = emg_signal_normalized

# Signal Windowing

In [ ]:
def window_signal(emg_signal, window_length, window_step):
    
    windows = []
    num_samples = len(emg_signal)

    for start in range(0, num_samples - window_length + 1, window_step):
        end = start + window_length

        # Extract window of the signal
        windowed_signal = emg_signal[start:end]

        # Append the window to the list
        windows.append(windowed_signal)

    return windows


# Feature Extraction

Recommended features:
- Mean absolute value (MAV)
- Standard deviation (STD)
- Variance (VAR)
- Waveform length (WL)
- Zero crossing (ZC)
- Root mean square (RMS)
- Number of peaks (NP)
- Slope sign change (SSC)
- Correlation coefficient (Cor)
- Integrated absolute value (IAV)

# Classification and model training